# 15. Writing a Simple Service and Client (Python)
http://wiki.ros.org/ROS/Tutorials/WritingServiceClient%28python%29

## setup environment

In [1]:
import os
import sys 

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level'

In [2]:
# get ROS related environment variables
envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  
del ROS_ENVs['PWD']

In [3]:
# define workspace path
ws_dir = 'catkin_ws'
ws_dir_abs = os.path.join(home_dir, ws_dir)
src_dir_abs = os.path.join(ws_dir_abs, 'src') 

# append environment paths
def gen_paths(ws_dir_abs, src_dir_abs):
    paths = {}
    paths['ROS_PACKAGE_PATH'] = src_dir_abs
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['ROSLISP_PACKAGE_DIRECTORIES'] = os.path.join(ws_dir_abs + 'devel/share/common-lisp')
    paths['PKG_CONFIG_PATH'] = os.path.join(ws_dir_abs + 'devel/lib/pkgconfig')
    paths['CMAKE_PREFIX_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel')
    return paths

def append_paths(ROS_ENVs, paths): 
    for path in ROS_ENVs.keys():
        os.environ[path] = ROS_ENVs[path]
        if path in paths.keys():
            os.environ[path] = '{}:{}'.format(paths[path], os.environ[path])

append_paths(ROS_ENVs, 
             gen_paths(ws_dir_abs, src_dir_abs))

In [4]:
# define package name and path
package_name = 'beginner_tutorials'

package_dir_abs = !rospack find {package_name}
package_dir_abs = package_dir_abs[0]
package_dir_abs

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials'

## Writing a Service Node
Here we'll create the service ("add_two_ints_server") node which will receive two ints and return the sum.

Change directory into the beginner_tutorials package, you created in the earlier tutorial, creating a package:

In [5]:
cd {package_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src/beginner_tutorials


Please make sure you have followed the directions in the previous tutorial for creating the service needed in this tutorial, creating the **AddTwoInts.srv** (be sure to choose the right version of build tool you're using at the top of wiki page in the link).

### The Code
Create the **scripts/add_two_ints_server.py** file within the beginner_tutorials package and paste the following inside it:

In [6]:
text = '''#!/usr/bin/env python

from beginner_tutorials.srv import *
import rospy

def handle_add_two_ints(req):
    print("Returning [%s + %s = %s]"%(req.a, req.b, (req.a + req.b)))
    return AddTwoIntsResponse(req.a + req.b)

def add_two_ints_server():
    rospy.init_node('add_two_ints_server')
    s = rospy.Service('add_two_ints', AddTwoInts, handle_add_two_ints)
    print("Ready to add two ints.")
    rospy.spin()

if __name__ == "__main__":
    add_two_ints_server()
'''

file_name = 'scripts/add_two_ints_server.py'

with open(file_name, 'w') as f:
    f.writelines(text)

Don't forget to make the node executable:

In [7]:
!chmod +x scripts/add_two_ints_server.py

### The Code Explained
Now, let's break the code down.

There's very little to writing a service using rospy. We declare our node using init_node() and then declare our service:  
```s = rospy.Service('add_two_ints', AddTwoInts, handle_add_two_ints)
```

This declares a new service named **add_two_ints** with the **AddTwoInts service type**. All requests are passed to **handle_add_two_ints** function. handle_add_two_ints is called with instances of **AddTwoIntsRequest** and returns instances of **AddTwoIntsResponse**.

Just like with the subscriber example, rospy.spin() keeps your code from exiting until the service is shutdown.

## Writing the Client Node
The Code
Create the scripts/add_two_ints_client.py file within the beginner_tutorials package and paste the following inside it:

In [8]:
text = '''#!/usr/bin/env python

import sys
import rospy
from beginner_tutorials.srv import *

def add_two_ints_client(x, y):
    rospy.wait_for_service('add_two_ints')
    try:
        add_two_ints = rospy.ServiceProxy('add_two_ints', AddTwoInts)
        resp1 = add_two_ints(x, y)
        return resp1.sum
    except rospy.ServiceException as e:
        print("Service call failed: %s"%e)

def usage():
    return "%s [x y]"%sys.argv[0]

if __name__ == "__main__":
    if len(sys.argv) == 3:
        x = int(sys.argv[1])
        y = int(sys.argv[2])
    else:
        print(usage())
        sys.exit(1)
    print("Requesting %s+%s"%(x, y))
    print("%s + %s = %s"%(x, y, add_two_ints_client(x, y)))
'''

file_name = 'scripts/add_two_ints_client.py'

with open(file_name, 'w') as f:
    f.writelines(text)

Don't forget to make the node executable:

In [9]:
!chmod +x scripts/add_two_ints_client.py

### The Code Explained
Now, let's break the code down.

The client code for calling services is also simple. For clients you don't have to call init_node(). We first call:
```
rospy.wait_for_service('add_two_ints')
```
This is a convenience method that **blocks until the service named add_two_ints is available**. Next we create a handle for calling the service:
```
add_two_ints = rospy.ServiceProxy('add_two_ints', AddTwoInts)
```
We can use this handle just like a normal ***function*** and call it:
```
resp1 = add_two_ints(x, y)
return resp1.sum
```

Because we've declared the type of the service to be **AddTwoInts**, it does the work of generating the **AddTwoIntsRequest** object for you (you're free to pass in your own instead). The return value is an **AddTwoIntsResponse** object. If the call fails, a **rospy.ServiceException** may be thrown, so you should setup the appropriate try/except block.

## Building your nodes
We use CMake as our build system and, yes, you have to use it even for Python nodes. This is to make sure that the autogenerated Python code for messages and services is created.

Go to your catkin workspace and run catkin_make.

In [10]:
cd {ws_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws


In [11]:
!catkin_make

Base path: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws
Source space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/src
Build space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/build
Devel space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/devel
Install space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/install
####
#### Running command: "make cmake_check_build_system" in "/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/build"
####
####
#### Running command: "make -j2 -l2" in "/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Beginner_Level/catkin_ws/build"
####
[  0%] Built target std_msgs_generate_messages_eus
[  0%] Built target _beginner_tutorials_generate_messages_check_deps_AddTwoInts
[  0%] Built target _beginner_tutorials_generate_messages_check_deps_Num
[  0%] Built target std_msgs_generate_messages_p